<a href="https://colab.research.google.com/github/DmitrijLeontev/Dmitrij_Leontev/blob/main/%D1%81%D1%82%D0%B0%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8%20/%D0%90%D0%B9%D0%A2%D0%B8%D0%9E%D0%BD_%D0%B7%D0%B0%D0%BC%D0%B5%D0%BD%D0%B0_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2%D1%8B%D1%85_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%BE%D0%B2/%D0%9F%D1%80%D0%BE%D0%B1%D1%83%D0%B5%D0%BC_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D1%81%D0%B5%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пробуем нейросеть

Обработка аудиосигналов: Параметризация аудио Импорт библиотек

В первую очередь импортируйте библиотеки и необходимые инструменты:

librosa – инструменты для работы с аудиосигналом; librosa.display – инструмент для визуализации аудио; IPython.display as ipd – проигрывание аудио в ноутбуке.

Сгенерируйте данные для обучения модели МОДЕЛЬ ПРЕОБРАЗОВАНИЯ TEXT-TO-SPEECH SILERO используется как предварительно обученная модель из PyTorch.

In [ ]:
# Подключаем гугл диск

# data_path = '/content/drive/MyDrive/Colab Notebooks/stagirovki/stagirov_robot/datasetitexzad/raspacovannie/golosblizko/dvigaisvper.WAV'

from google.colab import drive
drive.mount('/content/drive')

Обработка аудиосигналов: Параметризация аудио Импорт библиотек

В первую очередь импортируйте библиотеки и необходимые инструменты:

librosa – инструменты для работы с аудиосигналом; librosa.display – инструмент для визуализации аудио; IPython.display as ipd – проигрывание аудио в ноутбуке.

Загрузите необходимый датасет по ссылке при помощи gdown.download() и разархивируйте его:

In [ ]:
!pip install -q audiomentations

In [ ]:
!pip install -q cylimiter

In [ ]:
import gdown
import librosa as lb
import soundfile as sf
from IPython.display import Audio
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
from random import choice
import pickle

In [ ]:
# import functions and create object for transformation
from audiomentations import Compose, AddGaussianNoise, AirAbsorption, Limiter, RepeatPart, TanhDistortion, TimeMask

In [ ]:
from audiomentations import TimeStretch

In [ ]:
# https://drive.google.com/file/d/1kIZOkBIKIrLWqcgptDolg58rb5N1Q-vY/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=1kIZOkBIKIrLWqcgptDolg58rb5N1Q-vY', None, quiet=True)
!unzip -qo forward.zip

In [ ]:
# https://drive.google.com/file/d/10OIASzdWFzdOPrLJTSA5Ghd9ihfMHoQX/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=10OIASzdWFzdOPrLJTSA5Ghd9ihfMHoQX', None, quiet=True)
!unzip -qo backward.zip

In [ ]:
# https://drive.google.com/file/d/1KRusO1t6zrl2rprUjf2NBKwjxrj3V07j/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=1KRusO1t6zrl2rprUjf2NBKwjxrj3V07j', None, quiet=True)
!unzip -qo left.zip

In [ ]:
# https://drive.google.com/file/d/15Tim1Au-g5DP74RVQNGfpvqmziAueayI/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=15Tim1Au-g5DP74RVQNGfpvqmziAueayI', None, quiet=True)
!unzip -qo right.zip

In [ ]:
# https://drive.google.com/file/d/1owkxkb0gMvucD83Lx6VAmOKp_uvcw8gu/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=1owkxkb0gMvucD83Lx6VAmOKp_uvcw8gu', None, quiet=True)
!unzip -qo stop.zip

In [ ]:
!ls

In [ ]:
def transformer(folder, size=100):
    """
    функция принимает путь к каталогу и случайным образом берет из него аудио файл
    для генерации данных
    folder: путь к каталогу
    size: размер генерированного списка
    return: возвращает список генерированных команд размером size
    """
    augment = Compose([
    AddGaussianNoise(min_amplitude=0.01, max_amplitude=0.04, p=0.5),
    AirAbsorption(min_distance=300.0, max_distance=700.0, p=0.5),
    Limiter(min_threshold_db=-40.0, max_threshold_db=-0.0, threshold_mode="relative_to_signal_peak", p=0.5),
    TanhDistortion(min_distortion=0.01, max_distortion=0.7, p=0.5),
    TimeMask(min_band_part=0.1, max_band_part=0.2, fade=True, p=0.5),
    TimeStretch(min_rate=0.7, max_rate=1.5, leave_length_unchanged=True, p=0.5),
])
    data_x, data_y = list(), list()
    for _ in range(size):
        x, sr = lb.load(os.path.join(folder, choice(os.listdir(folder))))
        data = augment(x, sr)
        data_x.append(list(data))
        data_y.append(folders.index(folder))

    print(f'the length of the list x: {len(data_x)}, y: {len(data_y)}')
    return data_x, data_y

In [ ]:
train_x, train_y = list(), list()
folders = ['./forward', './backward', './left', './right', './stop']


for folder in folders:
    data_x, data_y = transformer(folder, 200)
    train_x.extend(data_x)
    train_y.extend(data_y)

In [ ]:
len(train_y), len(train_x), train_y[480]

In [ ]:
# with open('./data.pickle', 'wb') as f:
#     pickle.dump((train_x, train_y), f)

# with open('./data.pickle', 'rb') as f:
#     data_x, data_y = pickle.load(f)

In [ ]:
# https://drive.google.com/file/d/1q3pyc19MFE0Bhha_BYTUKnKicYigIxV-/view?usp=sharing
gdown.download('https://drive.google.com/uc?id=1q3pyc19MFE0Bhha_BYTUKnKicYigIxV-', None, quiet=True)

In [ ]:
with open('./data.pickle', 'rb') as f:
    data_x, data_y = pickle.load(f)

In [ ]:
len(data_x), len(data_y)

In [ ]:
Audio(data=data_x[676], rate=22050)

In [ ]:
idx = 828
print(data_y[idx], len(data_x[idx]))

In [ ]:
# постоянные переменные
SR = 22050 # частота дискретизации
HP = 200 # размер окна
FT = 500 # размер окна преобразования Фурье
CC = len(folders) # количество классов

In [ ]:
# Функция параметризации аудио

def get_features(y,                     # волновое представление сигнала
                 sr=SR,                   # частота дискретизации сигнала y
                 n_fft=FT,           # размер скользящего окна БПФ
                 hop_length=HP  # шаг скользящего окна БПФ
                 ):
    # Вычисление различных параметров (признаков) аудио

    # Среднеквадратическая амплитуда
    rmse = lb.feature.rms(y=y, hop_length=hop_length)
    # Спектральный центроид
    spec_cent = lb.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    # Ширина полосы частот
    spec_bw = lb.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    # Спектральный спад частоты
    rolloff = lb.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    # Пересечения нуля
    zcr = lb.feature.zero_crossing_rate(y, hop_length=hop_length)
    # Мел-кепстральные коэффициенты
    mfcc = lb.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    # Хромаграмма
    chroma_stft = lb.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)

    # Сборка параметров в общий список:
    # На один файл один усредненный вектор признаков
    features = {'rmse': rmse.mean(axis=1, keepdims=True),
                'spct': spec_cent.mean(axis=1, keepdims=True),
                'spbw': spec_bw.mean(axis=1, keepdims=True),
                'roff': rolloff.mean(axis=1, keepdims=True),
                'zcr' : zcr.mean(axis=1, keepdims=True),
                'mfcc': mfcc.mean(axis=1, keepdims=True),
                'stft': chroma_stft.mean(axis=1, keepdims=True)}

    return features

In [ ]:
# Функция объединения признаков в набор векторов

def stack_features(feat  # словарь признаков, отдельные векторы по ключу каждого признака
                   ):
    features = None
    for v in feat.values():
        features = np.vstack((features, v)) if features is not None else v

    return features.T.reshape(-1).tolist()

In [ ]:
# преобразуем полученные признаки в данные для обучения модели
X_data = np.array([stack_features(get_features(np.array(i))) for i in data_x]).astype('float32')
Y_data = to_categorical(data_y, CC)

print(type(X_data), type(Y_data), '\n', X_data.shape, Y_data.shape)

In [ ]:
# импортируем библиотеки для базовой модели
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=.2, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, sep='\n')

In [ ]:
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('clf', KNeighborsClassifier())
])

pipe.fit(X_train, y_train)

In [ ]:
Pipeline(steps=[('scaling', StandardScaler()), ('clf', KNeighborsClassifier())])

In [ ]:
# точность предсказания базовой модели
y_pred = pipe.predict(X_test)
accuracy_score(y_pred, y_test)

Подход номер три

Сгенерируйте данные для обучения модели МОДЕЛЬ ПРЕОБРАЗОВАНИЯ TEXT-TO-SPEECH SILERO используется как предварительно обученная модель из PyTorch.

Please follow the link to acquire more information about the model

In [ ]:
# вначале установите библиотеки для загрузки и сохранения предварительно обученной модели
!pip install -q torchaudio omegaconf
!pip install -q logmmse

# установить библиотеки для увеличения данных
!pip install -q audiomentations
!pip install -q cylimiter

In [ ]:
# import necessary libraries to generate voice command
import torch
import numpy as np
from logmmse import logmmse
from omegaconf import OmegaConf
from IPython.display import Audio
import librosa as lb
from librosa import feature as lbf
import gdown

# import necessary libraries for augmentation
import numpy as np
import os
from random import choice
# import functions for data augmentation and create object for transformation

In [ ]:
from audiomentations import Compose, AddGaussianNoise, AirAbsorption, Limiter, RepeatPart, TanhDistortion, TimeMask, TimeStretch

In [ ]:
# download and save the model in the variable 'model'
language = 'ru'
model_id = 'v4_ru'
device = torch.device('cpu')

model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)

In [ ]:
# create a function to generate voice command
def commandGenerator(size, command, model, speaker='random', sample_rate=24000):
    """
    the function takes desired size for generated data, text of a voice command,
    saved model to generate data, random speaker to overvoice command and
    frequency rate
    size: size of the final generated tensor
    command: text version of voice command
    speaker: voice of speaker
    sample_rate: frequency rate
    return: generated tensor based on function parameters
    """

    # create initial tensor with zeros and shape equals to 'size' parameter
    data = torch.zeros(size)

    for i in range(size[0]):
        # generate a sample
        audio = model.apply_tts(text=command,
                            speaker=speaker,
                            sample_rate=sample_rate)
        # enhance synthesis with logmmse
        audio = torch.tensor(logmmse(np.array(audio), sample_rate,
                                     output_file=None, initial_noise=1,
                                     window_size=160, noise_threshold=0.15))
        # calculate how many zeros to add in the beginning of the generated sample
        zeros_add = (size[1] - audio.size(0)) // 2
        # Create tensors of zeros for padding
        padding_start, padding_end = torch.zeros(zeros_add), torch.zeros(size[1] - audio.size(0) - zeros_add)
        # Concatenate tensors
        sample = torch.cat([padding_start, audio, padding_end])
        data[i] += sample

    return data if data.size(0) == size[0] and data.size(1) == size[1] else "Error Size"

In [ ]:
def augmentation(source, key, size=2500):
    """
    функция принимает сгенерированные данные из цифровых сетей в виде 2D-тензора
     и строка в качестве ключа для применения значений словарей
     источник: 2D-тензор
     ключ: ключ словаря
     вернуть data_x, data_y как список массивов, состоящих из дополненных данных
    """
    data_x, data_y = list(), list()
    while len(data_x) != size:

        augment = Compose([
            AddGaussianNoise(min_amplitude=0.01, max_amplitude=0.04, p=.5),
            AirAbsorption(min_distance=300.0, max_distance=700.0, p=.5),
            Limiter(min_threshold_db=-40.0, max_threshold_db=-0.0, threshold_mode="relative_to_signal_peak", p=.5),
            TanhDistortion(min_distortion=0.01, max_distortion=0.7, p=.5),
            TimeMask(min_band_part=0.1, max_band_part=0.2, fade=True, p=.5),
            TimeStretch(min_rate=0.7, max_rate=1.5, leave_length_unchanged=True, p=.5),
        ])

        try:
            sample = augment(source[np.random.randint(0, source.size(0))], 24000)
        except:
            continue
        else:
            data_x.append(sample), data_y.append(commands_list[key][0])

    return data_x, data_y



In [ ]:
# создать вспомогательные данные для создания набора данных train
rows = 100
commands_size = {'forward': (rows, 60000), 'backward': (rows, 60000), 'stop': (rows, 50000), 'left': (rows, 73000), 'right': (rows, 73000)}
commands_list = {'forward': (0, 'иди вперёд'), 'backward': (1, 'иди назад'), 'stop': (2, 'остановись'),
                 'left': (3, 'поворачивай влево'), 'right': (4, 'поворачивай вправо')}
# create lists to collect data right after an augmentation
X_data, Y_data = list(), list()

In [ ]:
# создайте данные для набора данных train с помощью функции «commandGenerator», используя цикл
for key in commands_size:
    print(commands_size[key], commands_list[key][1], commands_list[key][0])
    source = commandGenerator(commands_size[key], commands_list[key][1], model, speaker='random', sample_rate=24000)
    data_x, data_y = augmentation(source, key)
    print(len(data_x), len(data_y), data_y[0], data_y[-1])
    X_data.extend(data_x), Y_data.extend(data_y)

In [ ]:
# проверьте правильность данных и сохраните их
print(len(X_data), len(Y_data))

torch.save(Y_data, 'tensor_y.pt'), torch.save(X_data, 'tensor_x.pt')

In [ ]:
a = 10000
display(Audio(X_data[a], rate=24000)), Y_data[a]

In [ ]:
# Определите ссылку загрузчика файла на облачном хранилище Яндекс Диск
file_link = 'https://downloader.disk.yandex.ru/disk/c2478dcdbedf0f7c4d9a2f4472918f881e3ccee1082808f4c7b6f6213469f482/65553f37/fKqInKw3d7bLFOeFnMGnhNZsoTigsCIpYapgTpsof6zYW45Nj5YUVxW_mxlG5Ock8SFQlGtGuWGgj4_L4UpnQLSKSGLNlve4hFbmF5cZiRyr8npumZHI4midPdWhecNq?uid=1130000058358976&filename=tensor_x.pt&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=1130000058358976&fsize=1730339726&hid=d766bf1215d7ba8783d3ccbb6c7af7b9&media_type=compressed&tknv=v2&etag=b6b0a0d438adc2e7bd67362f1f334177'
# Загрузите файл с помощью gdown
output_path = '/content/commands_x'
gdown.download(file_link, output_path, quiet=True)

# Загрузите тензор PyTorch из загруженного файла
data_x = torch.load(output_path)

# Проверьте загруженный тензор

In [ ]:
# Определите ссылку загрузчика файла на облачном хранилище Яндекс Диск
file_link = 'https://downloader.disk.yandex.ru/disk/7ca8dc8dcb1a6ab171c99d76f3e60f801fc1e54b1989caa27e820f13c1491781/6555403b/fKqInKw3d7bLFOeFnMGnhNa4Akt7dfMo_GA1DeFMnTLRCRRoGbMbtdXURS8aTmjm58eZyPCGfAklXGXo-gyqn5Tzk9p7Ju-nKA7_IdpEqfur8npumZHI4midPdWhecNq?uid=1130000058358976&filename=tensor_y.pt&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=1130000058358976&fsize=25892&hid=2f4f6ab50d7ee2b8f33766e2c6786868&media_type=compressed&tknv=v2&etag=b1a729b6a797352739bfad624cf1cfe2'
#Загрузите файл с помощью gdown
output_path = '/content/commands_y'
gdown.download(file_link, output_path, quiet=True)

# Загрузите тензор PyTorch из загруженного файла.
data_y = torch.load(output_path)

# Проверьте загруженный тензор
print(len(data_y))

In [ ]:
a = 2490
display(Audio(data_x[a], rate=24000)), data_y[a]

Подход номер четыре

Features.ipynb

In [ ]:
import warnings # to suppress warnings
import numpy as np # to work with arrays
import pandas as pd # to work with dataframes
import torch # to work with tensors
from IPython.display import Audio # to play audio

import librosa as lb # to work with audio files
from librosa import feature as lbf # to extract features from audio files

import matplotlib.pyplot as plt # to plot graphs
import seaborn as sns # to plot graphs
sns.set(style='dark', palette='bright') # to set style and palette of graphs

from sklearn.model_selection import train_test_split # to split data into train and test sets
from sklearn.pipeline import Pipeline # to create a pipeline of transformers
from sklearn.compose import ColumnTransformer # to preprocess columns of a dataframe
from sklearn.base import BaseEstimator, TransformerMixin # to create custom transformers
from sklearn.preprocessing import MinMaxScaler # to scale features
from sklearn.model_selection import GridSearchCV # to perform grid search
from sklearn.metrics import accuracy_score # to calculate accuracy
from sklearn.linear_model import  LogisticRegression # to create a logistic regression model

from torch.utils.data import Dataset, DataLoader # to create a custom dataset of tensors
from torch import nn # to create neural network layers
import torchvision.transforms as transforms # to transform data
from torch import optim # to create an optimizer
from torch import sigmoid, relu, tanh # to create activation functions

# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install matplotlib-venn

In [ ]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
!pip install cartopy
import cartopy

In [ ]:
from feature_engine.selection import DropDuplicateFeatures # to drop duplicate features

In [ ]:
from feature_engine.selection import SmartCorrelatedSelection # to drop correlated features

Извлечение функций из голосовых команд

In [ ]:
# Загрузите набор данных в виде списков
# 0-2499: forward, 2500-4999: backward, 5000-7499: stop, 7500-9999: left, 10000-12499: right
# 0: forward, 1: backward, 2: stop, 3: left, 4: right
try:
    x = torch.load('tensor_x.pt')
    y = torch.load('tensor_y.pt')
except Exception as e:
    print(f"Error loading tensor: {e}")

In [ ]:
# проверьте набор данных после загрузки
idx = 2500
display(Audio(x[idx], rate=24000)), y[idx]

In [ ]:
# create a function to extract features from the audio

def get_features(y, sample_rate=24000, n_nfft=1024):
    '''
    the function extracts features from the audio signal
    y: audio signal
    sample_rate: sampling rate
    n_nfft: number of FFTs
    return: dictionary of features
    '''
    # extract spectral features
    # short-time Fourier transform
    chroma_stft = lbf.chroma_stft(y=y, sr=sample_rate, n_fft=n_nfft)
    # contstant-Q chromagram
    chroma_cqt = lbf.chroma_cqt(y=y, sr=sample_rate)
    # chroma energy normalized
    chroma_cens = lbf.chroma_cens(y=y, sr=sample_rate)
    # variable-Q chromagram
    chroma_vqt = lbf.chroma_vqt(y=y, sr=sample_rate, intervals='equal')
    # mel-scaled spectrogram
    melsp = lbf.melspectrogram(y=y, sr=sample_rate, n_fft=n_nfft)
    # mel-frequency cepstral coefficients
    mffc = lbf.mfcc(y=y, sr=sample_rate, n_fft=n_nfft)
    # root-mean-square value for each frame
    rmse = lbf.rms(y=y, frame_length=n_nfft, hop_length=n_nfft//2, center=True)
    # spectral centroid
    spec_cent = lbf.spectral_centroid(y=y, sr=sample_rate, n_fft=n_nfft, hop_length=n_nfft//2, center=True)
    # p'th-order spectral bandwidth
    spec_bw = lbf.spectral_bandwidth(y=y, sr=sample_rate, n_fft=n_nfft, hop_length=n_nfft//2, center=True, p=2)
    # spectral contrast
    spec_contrast = lbf.spectral_contrast(y=y, sr=sample_rate, n_fft=n_nfft, hop_length=n_nfft//2, center=True)
    # spectral flatness
    spec_flatness = lbf.spectral_flatness(y=y, n_fft=n_nfft, hop_length=n_nfft//2, center=True)
    # spectral roll-off frequency
    rolloff = lbf.spectral_rolloff(y=y, sr=sample_rate, n_fft=n_nfft, hop_length=n_nfft//2, center=True)
    # nth-order polynomial
    poly = lbf.poly_features(y=y, sr=sample_rate, n_fft=n_nfft, hop_length=n_nfft//2, center=True)
    # tonal centroid features
    tonnetz = lbf.tonnetz(y=y, sr=sample_rate)
    # zero-crossing rate
    zcr = lbf.zero_crossing_rate(y=y, frame_length=n_nfft, hop_length=n_nfft//2, center=True)

    # # extract rhythm features
    # # tempo (beats per minute)
    # tempo = lbf.tempo(y=y, sr=sample_rate)
    # # temprogram
    # tempogram = lbf.tempogram(y=y, sr=sample_rate, hop_length=n_nfft//2, win_length=n_nfft, center=True)
    # # fourier tempogram
    # ftempogram = lbf.fourier_tempogram(y=y, sr=sample_rate, hop_length=n_nfft//2, win_length=n_nfft, center=True)
    # # tempogram ratio features
    # temp_ratio = lbf.tempogram(y=y, sr=sample_rate, hop_length=n_nfft//2, win_length=n_nfft, center=True)

    # collect features in a dictionary
    features = {'stft': chroma_stft.mean(axis=1, keepdims=True),
                'cqt': chroma_cqt.mean(axis=1, keepdims=True),
                'cens': chroma_cens.mean(axis=1, keepdims=True),
                'vqt': chroma_vqt.mean(axis=1, keepdims=True),
                'melsp': melsp.mean(axis=1, keepdims=True),
                'mffc': mffc.mean(axis=1, keepdims=True),
                'rmse': rmse.mean(axis=1, keepdims=True),
                'spec_cent': spec_cent.mean(axis=1, keepdims=True),
                'spec_bw': spec_bw.mean(axis=1, keepdims=True),
                'spec_contrast': spec_contrast.mean(axis=1, keepdims=True),
                'spec_flatness': spec_flatness.mean(axis=1, keepdims=True),
                'rolloff': rolloff.mean(axis=1, keepdims=True),
                'poly': poly.mean(axis=1, keepdims=True),
                'tonnetz': tonnetz.mean(axis=1, keepdims=True),
                'zcr': zcr.mean(axis=1, keepdims=True),
                # 'tempo': tempo.reshape(1, -1),
                # 'tempogram': tempogram.mean(axis=1, keepdims=True),
                # 'ftempogram': ftempogram.mean(axis=1, keepdims=True),
                # 'temp_ratio': temp_ratio.mean(axis=1, keepdims=True)
                }
    return features

In [ ]:
# create a dictionary of features and their number
features_number = { key: value.shape[0] for key, value in get_features(x[0].numpy()).items() }
# create a list of columns names of each feature for the dataframe
features_columns = [ key if value == 1 else key + '_' + str(i) for key, value in features_number.items() for i in range(value) ]
features_columns[:7], len(features_columns)

In [ ]:
# create a function to join all features in one vector
def stack_features(feat):
    '''
    the function joins all features in one vector
    feat: dictionary of features
    return: list of features
    '''
    features = None
    for v in feat.values():
        features = np.vstack((features, v)) if features is not None else v

    return features.T.reshape(-1).tolist()

In [ ]:
# преобразуем полученные признаки в данные для обучения модели
X_data = np.array([stack_features(get_features(np.array(i))) for i in x]).astype('float32')

In [ ]:
# create a dataframe with features and target
df = pd.DataFrame(X_data, columns=features_columns)
df['target'] = y # add target column
# save the dataframe as a csv file
df.to_csv('features.csv', index=False, sep=',')

In [ ]:
df = pd.read_csv('features.csv').drop_duplicates() # load the dataframe and drop duplicates
df.reset_index(drop=True, inplace=True) # reset the index
df.shape # check the shape of the dataframe

In [ ]:
Базовая модель обучения для классификации голосовых команд

In [ ]:
plt.figure(figsize=(5, 2)) # set the size of the graph
ax = df.target.value_counts().sort_index().plot(kind='bar', title='Target distribution', rot=0) # plot the graph
ax.yaxis.grid(True) # show horizontal grid
plt.show() # show the graph